In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# ! pip install mxnet
# ! pip install gluonts

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.mx import DeepAREstimator, Trainer #, TrainerCallback, EarlyStoppingCallback
from gluonts.dataset.common import ListDataset

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
# np.random.seed(42)
# random.seed(42)

data = pd.read_csv('/content/gdrive/MyDrive/Thesis/DeepAR/clean_monthly_data.csv')
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [4]:
def format(df):
  df = df.set_index('Date')
  return df

MACRO

In [ ]:
macro = data.copy()

for m in ['GDP_growth',	'SP500_return',	'inflation_change',	'unemp_change',	'treasury_yield_change']:
  macro[f'{m}_lag'] = macro[m].shift(1)
  macro[f'{m}_lag2'] = macro[m].shift(2)
  macro[f'{m}_lag3'] = macro[m].shift(3)


macro['Quater_dummy'] = 0
macro['Quater_rippel'] = 0
macro['Date'] = pd.to_datetime(macro['Date'])
for index, row in data.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_rippel'] = 1

macro = format(macro)

In [ ]:
macro

,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,...,inflation_change_lag2,inflation_change_lag3,unemp_change_lag,unemp_change_lag2,unemp_change_lag3,treasury_yield_change_lag,treasury_yield_change_lag2,treasury_yield_change_lag3,Quater_dummy,Quater_rippel
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,...,NaN,NaN,-0.1,NaN,NaN,-0.141500,NaN,NaN,0,0
2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,...,1.000,NaN,-0.1,-0.1,NaN,-0.141500,-0.141500,NaN,0,0
2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,...,1.000,1.000,-0.2,-0.1,-0.1,-0.262978,-0.141500,-0.141500,0,0
2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,...,-0.100,1.000,0.2,-0.2,-0.1,-0.265995,-0.262978,-0.141500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,...,0.611,1.448,-0.1,-0.1,0.2,-0.092500,0.464452,0.621222,0,0
2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,...,0.392,0.611,-0.1,-0.1,-0.1,-0.274810,-0.092500,0.464452,0,0
2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,...,1.546,0.392,0.2,-0.1,-0.1,-0.084690,-0.274810,-0.092500,0,0


In [5]:
def data_prep(df, ticker):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-6],
                "target": df[ticker][-6:].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [6]:
hyperparameters = {
    "prediction_length": 6,
    "context_length": 8,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}

In [7]:
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

In [8]:
def fit_n_predict(model, training, test):
  predictor = model.train(training)
  predictor = next(predictor.predict(test))
  samples = predictor.samples
  time = predictor.index
  return samples, time

In [9]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)
    #print(df)

    t_loss = 0

    for i in df.index:
        t_loss += (alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i]) +
                 (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i])

    return t_loss

In [10]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [ ]:
def get_performance(ticker, dates):

  dta = macro.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:6]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  t_loss = t_loss/7

  return t_loss

In [11]:
tickers = ['NVS', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG']

In [ ]:
#for t in tickers:
for t in ['MRK', 'NVO', 'PFE', 'ROG']:
  loss = get_performance(t, dates6m)
  print(f'{t}: {loss}')

100%|██████████| 50/50 [00:01<00:00, 44.47it/s, epoch=30/30, avg_epoch_loss=3.29]


MRK: 3.5995152725557595


100%|██████████| 50/50 [00:00<00:00, 61.84it/s, epoch=30/30, avg_epoch_loss=6.01]


NVO: 4.789551815378274


100%|██████████| 50/50 [00:00<00:00, 60.55it/s, epoch=30/30, avg_epoch_loss=3.26]


PFE: 4.799998616816317


100%|██████████| 50/50 [00:00<00:00, 63.79it/s, epoch=30/30, avg_epoch_loss=3.91]

ROG: 47.46697247671006


STOCKS

In [11]:
stocks = data.copy()

for t in tickers:
  stocks[f'{t}_lag'] = stocks[t].shift(1)
  stocks[f'{t}_lag2'] = stocks[t].shift(2)
  #stocks[f'{t}_lag3'] = stocks[t].shift(3)


stocks['Quater_dummy'] = 0
stocks['Quater_rippel'] = 0
stocks['Date'] = pd.to_datetime(stocks['Date'])
for index, row in stocks.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        stocks.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        stocks.loc[index, 'Quater_rippel'] = 1

stocks = format(stocks)

In [12]:
def data_prep(df, ticker):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-6],
                "target": df[ticker][-6:].values,
                "dynamic_feat": [
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [13]:
def get_performance(ticker, dates):

  dta = stocks.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:6]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  t_loss = t_loss/7

  return t_loss

In [ ]:
#for t in ['NVS', 'AZN', 'BMY', 'JNJ', 'LLY']:
for t in ['MRK', 'NVO', 'PFE', 'ROG']:
  loss = get_performance(t, dates6m)
  print(f'{t}: {loss}')

100%|██████████| 50/50 [00:01<00:00, 44.14it/s, epoch=30/30, avg_epoch_loss=3.39]


MRK: 3.5166064824016425


100%|██████████| 50/50 [00:00<00:00, 62.16it/s, epoch=30/30, avg_epoch_loss=3.38]


NVO: 4.447450386273468


100%|██████████| 50/50 [00:00<00:00, 63.52it/s, epoch=30/30, avg_epoch_loss=3.24]


PFE: 5.375644164394367


 56%|█████▌    | 28/50 [00:00<00:00, 38.74it/s, epoch=12/30, avg_epoch_loss=3.81]
/usr/local/lib/python3.10/dist-packages/gluonts/mx/trainer/_base.py:484: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(
  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
all = data.copy()

for m in ['GDP_growth',	'SP500_return',	'inflation_change',	'unemp_change',	'treasury_yield_change']:
  all[f'{m}_lag'] = all[m].shift(1)
  all[f'{m}_lag2'] = all[m].shift(2)
  all[f'{m}_lag3'] = all[m].shift(3)


all['Quater_dummy'] = 0
all['Quater_rippel'] = 0
all['Date'] = pd.to_datetime(all['Date'])
for index, row in all.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        all.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        all.loc[index, 'Quater_rippel'] = 1


for t in tickers:
  all[f'{t}_lag'] = all[t].shift(1)
  all[f'{t}_lag2'] = all[t].shift(2)

In [13]:
def data_prep(df, ticker):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-6],
                "target": df[ticker][-6:].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [14]:
def get_performance(ticker, dates):

  dta = all.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:6]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  t_loss = t_loss/7

  return t_loss

In [15]:
all = format(all)

In [16]:
#for t in ['NVS', 'AZN', 'BMY', 'JNJ', 'LLY']:
for t in ['MRK', 'NVO', 'PFE', 'ROG']:
  loss = get_performance(t, dates6m)
  print(f'{t}: {loss}')

100%|██████████| 50/50 [00:00<00:00, 67.99it/s, epoch=30/30, avg_epoch_loss=3.33]


MRK: 3.5802624263573373


100%|██████████| 50/50 [00:00<00:00, 57.53it/s, epoch=30/30, avg_epoch_loss=3.35]


NVO: 4.421962043290086


100%|██████████| 50/50 [00:00<00:00, 63.25it/s, epoch=30/30, avg_epoch_loss=3.25]


PFE: 4.495839561586715


100%|██████████| 50/50 [00:00<00:00, 62.66it/s, epoch=30/30, avg_epoch_loss=5.2]

ROG: 47.51815299320999
